In [3]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import statsmodels.api as sm
from sklearn.cluster import KMeans
from factor_analyzer import FactorAnalyzer


In [78]:
pq = pd.read_parquet("price_top290.parquet")
pq['date'] = pq.index
pq1 = pq[pq.index == "2023-01-03"]
pq1["vol-value"] = pq1['Close'] * pq1['Volume']
list_ = pq1.sort_values(by='vol-value', ascending=False)[:]['ticker'].to_list()


C:\Users\cwcha\AppData\Local\Temp\ipykernel_27208\398339267.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pq1["vol-value"] = pq1['Close'] * pq1['Volume']


In [79]:
df_close_unpivoted = pq[pq['ticker'].isin(list_)]
df_close_pivoted = pd.pivot_table(data=df_close_unpivoted, values='Close', index='date', columns='ticker')
df_close_pivoted = df_close_pivoted[df_close_pivoted.index <= "2023-01-01"]
df_close_pivoted

ticker,0050,0056,006208,00633L,00637L,00720B,00753L,1101,1216,1303,...,6741,6770,6806,8069,8171,8299,8436,9904,9910,9921
date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,74.050003,23.830000,41.169998,27.290001,10.32,37.619999,19.340000,28.723679,68.300003,73.000000,...,NaN,NaN,NaN,30.250000,9.928683,228.5,445.606812,32.200001,126.826302,142.885651
2019-01-03,73.349998,23.590000,40.779999,27.590000,10.44,37.950001,19.100000,28.440487,67.500000,73.300003,...,NaN,NaN,NaN,30.299999,10.065946,227.0,455.180389,33.000000,128.855515,154.792786
2019-01-04,72.199997,23.400000,40.020000,28.530001,10.81,37.889999,19.559999,28.683222,67.900002,73.099998,...,NaN,NaN,NaN,29.000000,9.882929,224.5,469.975922,33.400002,130.884735,152.312134
2019-01-07,73.949997,23.730000,41.070000,28.670000,10.87,37.709999,20.290001,29.128237,70.400002,75.099998,...,NaN,NaN,NaN,29.600000,9.974437,228.0,463.883667,33.900002,133.252167,159.257965
2019-01-08,73.650002,23.790001,40.900002,28.660000,10.85,37.930000,20.480000,28.764133,70.199997,74.199997,...,NaN,NaN,NaN,29.799999,10.065946,230.5,456.921051,34.200001,133.252167,154.792786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,111.099998,25.580000,62.950001,38.590000,15.18,35.709999,10.850000,34.400002,65.800003,71.400002,...,105.5,32.750000,85.599998,166.000000,47.858646,305.5,175.000000,33.700001,187.500000,208.000000
2022-12-27,111.599998,25.629999,63.099998,39.209999,15.47,35.599998,11.130000,34.299999,66.199997,71.699997,...,112.0,33.000000,87.699997,168.000000,47.809959,315.5,177.500000,33.900002,185.267853,208.500000
2022-12-28,110.000000,25.379999,62.099998,39.250000,15.49,35.070000,11.270000,34.299999,66.199997,71.400002,...,109.0,32.150002,88.900002,161.000000,48.394196,310.5,171.000000,33.799999,187.053574,204.500000


In [80]:
stock_list = list_
# stock_list = etfs

spx_close = yf.download("^TWII", start=pd.to_datetime("2019-01-01"), end="2023-01-01", progress=False)['Adj Close']
spx_close.head()

Date
2019-01-02    9554.139648
2019-01-03    9492.419922
2019-01-04    9382.509766
2019-01-07    9590.299805
2019-01-08    9563.599609
Name: Adj Close, dtype: float64

In [81]:
df_close = df_close_pivoted
df_close_pct = df_close.pct_change().iloc[1:, :]
spx_pct = spx_close.pct_change()

df_close_pct.dropna(inplace=True, axis=1)
spx_pct.dropna(inplace=True)

df_close_pct_m = df_close_pct.resample("M").agg(lambda x: (x+1).prod() - 1)
spx_pct_m = spx_pct.resample("M").agg(lambda x: (x+1).prod() - 1)

In [82]:
factor = 4
res_df = pd.DataFrame()
alpha_dict = {}
X = spx_pct_m
X_sm = sm.add_constant(X)
for ticker in df_close_pct_m.columns:
    # OLS regression
    close = df_close_pct_m[ticker]
    y = close
    model = sm.OLS(y, X_sm)

    res = model.fit()
    residuals = res.resid
    res_df[ticker] = residuals

    # store alpha
    res_table = res.summary().tables
    alpha = float(res_table[1].data[1][1].strip())
    alpha_dict[ticker] = alpha
factor_analyzer = FactorAnalyzer(n_factors=factor, rotation='promax') # varimax
factor_analyzer.fit(res_df.corr())

# Get factor loadings
factor_loadings = factor_analyzer.loadings_
df = pd.DataFrame(factor_loadings, index=list(res_df.columns))
df['Ticker'] = df.index
# Apply K-means clustering
kmeans = KMeans(n_clusters=factor)
kmeans.fit(df.iloc[:, :factor])

# Assign cluster labels to tickers
df['Cluster'] = kmeans.labels_

# Display the result
res = df[['Ticker', 'Cluster']]
res = res.copy()
# add clustered value to check further
cluster_value = []
for i in range(len(res)):
    classification = res.iloc[i, 1]
    cluster_value.append(df.iloc[i, classification])
res['Cluster_value'] = cluster_value

sorted_res = res.sort_values(by='Cluster')


res_dict = {}
for i in range(factor):

    res_dict["class" + str(i)] = list(sorted_res[sorted_res['Cluster'] == i]['Ticker'])
res_dict
# factor == 3比較穩定 但還是不合理

{'class0': ['2884',
  '9910',
  '3083',
  '3218',
  '3293',
  '3479',
  '3708',
  '4114',
  '4130',
  '4167',
  '4743',
  '4746',
  '6180',
  '6491',
  '6547',
  '8171',
  '8436',
  '9904',
  '2201',
  '1795',
  '9921',
  '00753L',
  '0056',
  '1101',
  '00633L',
  '1476',
  '1701',
  '1477',
  '1529',
  '00637L',
  '1514',
  '1513'],
 'class1': ['6118',
  '2912',
  '3008',
  '6111',
  '3443',
  '1536',
  '3680',
  '5425',
  '1319',
  '1216',
  '3406',
  '6143',
  '3587',
  '4977',
  '2313',
  '2317',
  '2327',
  '8299',
  '4958',
  '2388',
  '6533',
  '6531',
  '00720B',
  '2498',
  '6446',
  '6414',
  '6279',
  '2880',
  '8069'],
 'class2': ['006208',
  '5274',
  '6223',
  '6679',
  '6409',
  '5347',
  '6669',
  '0050',
  '3707',
  '3034',
  '1590',
  '2303',
  '2308',
  '2357',
  '2382',
  '4966',
  '2454'],
 'class3': ['1303',
  '1810',
  '2027',
  '2409',
  '2412',
  '2465',
  '2609',
  '2610',
  '2618',
  '2881',
  '2882',
  '2883',
  '2891',
  '1326',
  '2603',
  '1605']}

In [83]:
import numpy as np
indexes = pd.DataFrame()
for key, value in res_dict.items():
    print(value)
    index_close = df_close[value]
    index_close = index_close/len(value)
    indexes[f"{str(key)}-index"] = index_close.sum(axis=1)

    indexes_pct = indexes.pct_change()
    indexes_pct.dropna(inplace=True)

    indexes_pct_m = indexes_pct.resample("M").agg(lambda x: (x+1).prod() - 1)
    mean = np.mean(indexes_pct_m, axis=0)
    sd = np.std(indexes_pct_m, axis=0)
    reg_coef = []
    for i in range(len(indexes_pct_m.columns)):
        X = spx_pct_m
        X_sm = sm.add_constant(X)
        y = indexes_pct_m.iloc[:, i]
        model = sm.OLS(y, X_sm)
        res = model.fit()
        reg_coef.append(res.summary().tables[1].data[2][1].replace(" ", ""))


result = pd.DataFrame()
result.index = list(indexes.columns)
result["mean"] = mean
result["sd"] = sd
result["reg_coef"] = reg_coef
result.sort_values(by='reg_coef', ascending=False)

['2884', '9910', '3083', '3218', '3293', '3479', '3708', '4114', '4130', '4167', '4743', '4746', '6180', '6491', '6547', '8171', '8436', '9904', '2201', '1795', '9921', '00753L', '0056', '1101', '00633L', '1476', '1701', '1477', '1529', '00637L', '1514', '1513']
['6118', '2912', '3008', '6111', '3443', '1536', '3680', '5425', '1319', '1216', '3406', '6143', '3587', '4977', '2313', '2317', '2327', '8299', '4958', '2388', '6533', '6531', '00720B', '2498', '6446', '6414', '6279', '2880', '8069']
['006208', '5274', '6223', '6679', '6409', '5347', '6669', '0050', '3707', '3034', '1590', '2303', '2308', '2357', '2382', '4966', '2454']
['1303', '1810', '2027', '2409', '2412', '2465', '2609', '2610', '2618', '2881', '2882', '2883', '2891', '1326', '2603', '1605']


,mean,sd,reg_coef
class1-index,0.007895,0.087155,1.2109
class3-index,0.013863,0.105430,1.2021
class2-index,0.022883,0.078555,1.1001
class0-index,0.014735,0.076383,0.9094


In [84]:
import json
with open(f"real_volume_for2023_top100_cluster9_123.json", "w") as f:
    json.dump(res_dict, f, indent=4)